In [1]:
def csv_dump(rows, filename, append=False, delimiter=";"):
    import csv
    open_param = 'a' if append else 'w'
    with open(filename, open_param) as csvfile:
        writer = csv.writer(csvfile, delimiter=delimiter, quotechar=str('"'), quoting=csv.QUOTE_MINIMAL)
        rows = [list(row) for row in rows]  # Like copy, but converts inner tuples to lists
        writer.writerows(rows)

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
from collections import defaultdict

df = pd.read_csv('/home/shakurova-ksenia/Work/hackaton/hackprague_txs.csv', sep=';')#, nrows=100)
df.head(10)

/home/shakurova-ksenia/.pyenv/versions/3.7.6/envs/strv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,transaction_id,tx_date,client_id,client_gender,client_year_of_birth,amount,merchant_uid,merchant_category,shop_tags,merchant_category_id,shop_uid,shop_type,region,country
0,1,2020-07-02,146167,Female,1980.0,-594,X0noQaM22AHzkPN9erAb74,Professional Services,{Delivery Service},1483998.0,zYMe28Vm9LFEpAN18vaQvG,MERCHANT,Pardubice,CZ
1,2,2020-06-11,40161,Female,1980.0,-986,X0noQaM22AHzkPN9erAb74,Professional Services,{Delivery Service},1483998.0,zYMe28Vm9LFEpAN18vaQvG,MERCHANT,Pardubice,CZ
2,3,2020-08-14,200977,Female,1965.0,-102,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
3,4,2020-06-10,200977,Female,1965.0,-158,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
4,5,2020-05-27,200977,Female,1965.0,-126,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
5,6,2020-10-16,73521,Female,1965.0,-444,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
6,7,2020-10-13,10490,Male,1985.0,-252,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
7,8,2020-09-26,105490,Female,1975.0,-1365,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
8,9,2020-09-21,14293,Male,1965.0,-150,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
9,10,2020-09-21,71421,Female,1995.0,-315,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ


In [3]:
df.shape

(13511090, 14)

In [4]:
df = df[~df.merchant_category.isnull()]
df = df[~df.shop_tags.isnull()]
df.shape

(12633244, 14)

In [5]:
print(df[df.region == 'Pardubice'].shape)
print(df[df.region == 'Kladno'].shape)
print(df[df.region == 'Hlavní město Praha'].shape)

(143456, 14)
(136656, 14)
(2838721, 14)


In [6]:
# We need only Prague
df = df[df.region == 'Hlavní město Praha']
df.shape

(2838721, 14)

In [7]:
df = df[df.amount < 0]
df.shape

(2838591, 14)

In [8]:
df.head(100)

,transaction_id,tx_date,client_id,client_gender,client_year_of_birth,amount,merchant_uid,merchant_category,shop_tags,merchant_category_id,shop_uid,shop_type,region,country
1978,1979,2020-10-16,194567,Male,1960.0,-982,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1979,1980,2020-10-10,54153,Female,1995.0,-515,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1980,1981,2020-10-14,37299,Male,1985.0,-614,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1982,1983,2020-10-07,194567,Male,1960.0,-967,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1984,1985,2020-09-29,67336,Male,1965.0,-2300,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,2195,2020-06-11,105126,Female,1975.0,-720,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
2195,2196,2020-05-08,103492,Female,1975.0,-659,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
2196,2197,2020-05-29,103492,Female,1975.0,-466,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
2197,2198,2020-06-19,103492,Female,1975.0,-1918,ERj1R2KVoNt5L7gyR1a9GA,Health,"{Clinic,Rehabilitation}",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ


## Gender

In [10]:
import json   

In [11]:
gender_mapping = {
    'Female': 1,
    'Male': 2  
}
with open("/home/shakurova-ksenia/Work/hackaton/mapping_genders.json", "w") as outfile:  
    json.dump(gender_mapping, outfile) 


In [12]:
df['client_gender'] = df['client_gender'].apply(lambda x: gender_mapping[x]) 


## merch categoris

In [13]:
merchant_categories = set(df.merchant_category)
categories_mapping = dict([(c, i) for i, c in enumerate(merchant_categories)])

with open("/home/shakurova-ksenia/Work/hackaton/mapping_categories.json", "w") as outfile:  
    json.dump(categories_mapping, outfile) 
categories_mapping

{'Beauty': 0,
 'Children': 1,
 'Health': 2,
 'Groceries': 3,
 'Drugstore': 4,
 'Digital Services': 5,
 'Car': 6,
 'Sport': 7,
 'Pets': 8,
 'Fashion': 9,
 'Electronics': 10,
 'Consumer Goods': 11,
 'Bills And Household': 12,
 'Professional Services': 13,
 'Shopping Online': 14,
 'Food And Drink': 15,
 'Financial Services': 16,
 'Atm': 17,
 'House And Garden': 18,
 'Freetime': 19,
 'Travel': 20,
 'Culture': 21}

In [14]:
df['merchant_category'] = df['merchant_category'].apply(lambda x: categories_mapping[x]) 

## merch tags 

In [17]:
def trans_tags(x):
    shop_tags = [t.strip() for t in x[0][1:-1].strip().split(',')]
    return shop_tags

In [18]:
a = df['shop_tags'].apply(lambda x: trans_tags([x]))
a.head()

1978    [Clinic, Rehabilitation]
1979    [Clinic, Rehabilitation]
1980    [Clinic, Rehabilitation]
1982    [Clinic, Rehabilitation]
1984    [Clinic, Rehabilitation]
Name: shop_tags, dtype: object

In [19]:
df['shop_tags'] = a

In [22]:
shop_tags_set = list()
cat_tags_dict = defaultdict(set)
for idx, row in df[~df['shop_tags'].isnull()].iterrows():
    shop_tags = row['shop_tags']
    shop_tags_set.extend(shop_tags)
    cat_tags_dict[row['merchant_category']].update(shop_tags)


In [23]:
shop_tags_set = set(shop_tags_set)
len(shop_tags_set)

301

In [24]:
cat_tags_dict

defaultdict(set,
            {2: {'Aesthetic Medicine',
              'Alternative Medicine',
              'Clinic',
              'Dental Care',
              'Dentistry',
              'Drugstore',
              'Fashion Accessories',
              'Footwear',
              'General Practitioner',
              'Gynecology',
              'Health Supplies',
              'Hospital',
              'Online Health',
              'Ophthalmologist',
              'Optics And Optical Services',
              'Pharmacy',
              'Rehabilitation',
              'Reproductive Medicine',
              'Shopping Online',
              'Sunglasses'},
             15: {'Accommodations',
              'Alcohol',
              'American Cuisine',
              'American Fast Food',
              'Arabic Cuisine',
              'Asian Cuisine',
              'Bageterie',
              'Bakery',
              'Balkan Cuisine',
              'Bar',
              'Beer / Cider',
              '

In [25]:
tags_mapping = dict([(c, i) for i, c in enumerate(shop_tags_set)])

with open("/home/shakurova-ksenia/Work/hackaton/mapping_tags.json", "w") as outfile:  
    json.dump(tags_mapping, outfile) 
tags_mapping

{'Fish': 0,
 'Czech Restaurant': 1,
 'Air Tickets': 2,
 'Modeling': 3,
 'Erotic Goods': 4,
 'Driving School': 5,
 'Fashion Accessories': 6,
 'Martial Arts': 7,
 'Taxi': 8,
 'Ice Cream': 9,
 'Subscription': 10,
 'Building Supply Store': 11,
 'Wine': 12,
 'Bike': 13,
 'Butcher Shop': 14,
 'Services - Mix': 15,
 'Dance': 16,
 'Motorcycle Accessories': 17,
 'Beer / Cider': 18,
 'Aviation': 19,
 'Healthy Food And Farm Products': 20,
 'Photo Printing': 21,
 'Delivery Service': 22,
 'Garden': 23,
 'Cafe': 24,
 'Gourmet': 25,
 'Canoe Equipment': 26,
 'Bike Service': 27,
 'Delicatessen And Specialized Stores': 28,
 'Reproductive Medicine': 29,
 'Asian Cuisine': 30,
 'Equipment Cosmetics': 31,
 'Leather Haberdashery': 32,
 'Pantyhose / Socks': 33,
 'Pc Service': 34,
 'Beauty Salon': 35,
 'Car Sales': 36,
 'Gas And Electricity': 37,
 'Zero Waste': 38,
 'Weapons And Ammunition': 39,
 'Alternative Medicine': 40,
 'Fashion Chain - Mix': 41,
 'Ophthalmologist': 42,
 'Fruit/Vegetables': 43,
 'Sweetsho

In [26]:
cat_tags_dict_int = {cat: [tags_mapping[tag] for tag in tags] for cat,tags in  cat_tags_dict.items()}
cat_tags_dict_int

{2: [6,
  238,
  166,
  242,
  95,
  178,
  180,
  29,
  118,
  267,
  40,
  42,
  127,
  202,
  210,
  291,
  219,
  175,
  298,
  70],
 15: [227,
  1,
  0,
  160,
  228,
  230,
  78,
  235,
  9,
  236,
  237,
  168,
  12,
  88,
  89,
  243,
  90,
  14,
  91,
  93,
  18,
  250,
  176,
  177,
  252,
  102,
  24,
  182,
  25,
  105,
  256,
  255,
  28,
  30,
  110,
  112,
  115,
  116,
  124,
  44,
  126,
  45,
  49,
  133,
  51,
  283,
  54,
  210,
  214,
  290,
  59,
  218,
  149,
  150,
  152,
  64,
  68],
 11: [159,
  73,
  74,
  4,
  232,
  233,
  10,
  15,
  98,
  96,
  252,
  21,
  179,
  24,
  183,
  184,
  28,
  108,
  109,
  192,
  199,
  194,
  271,
  130,
  51,
  53,
  283,
  210,
  137,
  139,
  145,
  62,
  224,
  225],
 17: [157, 122],
 3: [0,
  78,
  233,
  236,
  12,
  89,
  92,
  14,
  18,
  176,
  177,
  20,
  180,
  25,
  24,
  183,
  28,
  109,
  258,
  112,
  261,
  117,
  38,
  43,
  44,
  126,
  45,
  201,
  132,
  133,
  51,
  283,
  284,
  136,
  210,
  290,
  

In [27]:
with open("/home/shakurova-ksenia/Work/hackaton/categories_to_tags.json", "w") as outfile:  
    json.dump(cat_tags_dict_int, outfile) 

In [46]:
df['shop_tags'] = df['shop_tags'].apply(lambda x: [tags_mapping[x_] for x_ in x])

In [48]:
df.amount = df.amount.apply(lambda x: x*-1)
df.head()

,transaction_id,tx_date,client_id,client_gender,client_year_of_birth,amount,merchant_uid,merchant_category,shop_tags,merchant_category_id,shop_uid,shop_type,region,country
1978,1979,2020-10-16,194567,2,1960.0,982,ERj1R2KVoNt5L7gyR1a9GA,2,"[178, 242]",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1979,1980,2020-10-10,54153,1,1995.0,515,ERj1R2KVoNt5L7gyR1a9GA,2,"[178, 242]",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1980,1981,2020-10-14,37299,2,1985.0,614,ERj1R2KVoNt5L7gyR1a9GA,2,"[178, 242]",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1982,1983,2020-10-07,194567,2,1960.0,967,ERj1R2KVoNt5L7gyR1a9GA,2,"[178, 242]",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ
1984,1985,2020-09-29,67336,2,1965.0,2300,ERj1R2KVoNt5L7gyR1a9GA,2,"[178, 242]",155.0,9BjNVjpBBjCerro5EKqA4q,MERCHANT,Hlavní město Praha,CZ


In [50]:
df.shape

(2838591, 14)

In [51]:
df.to_csv('final_data.csv', sep=";")

# Shops , coords and districts

In [31]:
min(df.amount), max(df.amount), np.mean(df.amount)

(-345413, -1, -650.3165183008049)

In [33]:
from collections import Counter
Counter(df.client_gender)

Counter({2: 1610606, 1: 1227985})

In [42]:
min(df.client_year_of_birth), max(df.client_year_of_birth), np.mean(df.client_year_of_birth)

(1930.0, 2010.0, 1981.4868320140747)

In [70]:
int(4.4)

4

In [43]:
len(set(df.shop_uid))

6355

In [52]:
len(set(df.client_id))

37382

In [69]:
df.shape

(2838591, 14)

In [83]:
districts = ["Praha 1",
"Praha 2",
"Praha 3",
"Praha 4",
"Praha 5",
"Praha 6",
"Praha 7",
"Praha 8",
"Praha 9",
"Praha 10",
"Praha 11",
"Praha 12",
"Praha 13",
"Praha 14",
"Praha 15",
"Praha 16",
"Praha 17",
"Praha 18",
"Praha 19",
"Praha 20",
"Praha 21",
"Praha 22",
"Praha-Bøezinìves",
"Praha-Benice",
"Praha-Bìchovice",
"Praha-Dolní Chabry",
"Praha-Dolní Poèernice",
"Praha-Dolní Mìcholupy",
"Praha-Dubeè",
"Praha-Královice",
"Praha-Kunratice",
"Praha-Kolodìje",
"Praha-Kolovraty",
"Praha-Køeslice",
"Praha-Klánovice",
"Praha-Libuš",
"Praha-Lochkov",
"Praha-Lysolaje",
"Praha-Lipence",
"Praha-Nebušice",
"Praha-Nedvìzí",
"Praha-Petrovice",
"Praha-Pøední Kopanina",
"Praha-Suchdol",
"Praha-Slivenec",
"Praha-Satalice",
"Praha-Troja",
"Praha-Vinoø",
"Praha-Velká Chuchle",
"Praha-Zbraslav",
"Praha-Zlièín",
"Praha-Újezd",
"Praha-Èakovice",
"Praha-Ïáblice",
"Praha-Øeporyje",
"Praha-Šeberov",
"Praha-Štìrboholy"]

In [100]:
praha_mapping = dict([(c, i) for i, c in enumerate(districts)])

with open("/home/shakurova-ksenia/Work/hackaton/mapping_praha.json", "w") as outfile:  
    json.dump(praha_mapping, outfile) 

In [71]:
districts_size_dict = {'Praha 1': 1188,
'Praha 2': 636,
'Praha 3': 636,
'Praha 4': 636,
'Praha 5': 636,
'Praha 6': 636,
'Praha 7': 636,
'Praha 8': 636,
'Praha 9': 28,
'Praha 10': 28,
'Praha 11': 27,
'Praha 12': 26,
'Praha 13': 26,
'Praha 14': 25,
'Praha 15': 25,
'Praha 16': 24,
'Praha 17': 24,
'Praha 18': 23,
'Praha 19': 22,
'Praha 20': 22,
'Praha 21': 21,
'Praha 22': 21,
'Praha-Bøezinìves': 20,
'Praha-Benice': 20,
'Praha-Bìchovice': 19,
'Praha-Dolní Chabry': 19,
'Praha-Dolní Poèernice': 18,
'Praha-Dolní Mìcholupy': 17,
'Praha-Dubeè': 17,
'Praha-Královice': 16,
'Praha-Kunratice': 16,
'Praha-Kolodìje': 15,
'Praha-Kolovraty': 15,
'Praha-Køeslice': 14,
'Praha-Klánovice': 13,
'Praha-Libu': 13,
'Praha-Lochkov': 12,
'Praha-Lysolaje': 12,
'Praha-Lipence': 11,
'Praha-Nebuice': 11,
'Praha-Nedvìzí': 10,
'Praha-Petrovice': 10,
'Praha-Pøední Kopanina': 9,
'Praha-Suchdol': 8,
'Praha-Slivenec': 8,
'Praha-Satalice': 7,
'Praha-Troja': 7,
'Praha-Vinoø': 6,
'Praha-Velká Chuchle': 6,
'Praha-Zbraslav': 5,
'Praha-Zlièín': 4,
'Praha-Újezd': 4,
'Praha-Èakovice': 3,
'Praha-Ïáblice': 3,
'Praha-Øeporyje': 2,
'Praha-eberov': 2,
'Praha-tìrboholy': 1}

In [92]:
shops_ids = set(df.shop_uid) 
len(shops_ids)

6355

In [93]:
import random

random.shuffle(list(shops_ids))
len(shops_ids), type(shops_ids)


(6355, set)

In [84]:
len(districts)

57

In [89]:
with open('shop_points.json') as input_file:
    shops_coords = json.load(input_file)

In [90]:
shops_coords = shops_coords['result']


In [96]:
cur = 0
for distr_info in shops_coords:
    distr_info['shops_uid'] = list(shops_ids)[cur:cur+distr_info['shop_num']]
    cur += distr_info['shop_num']
    
#     print(len(distr_shops),distr_info['shop_num'])

In [ ]:
cur = 0
for i, distr in enumerate(districts):
    for shop in shops_ids[cur:cur+districts_size_dict[distr]]

In [97]:
shops_coords

[{'id': 0,
  'prague_part': 'Praha 1',
  'shop_num': 1188,
  'shops_coords': [[14.41509605946885, 50.08002856986629],
   [14.420741616347616, 50.08968401939787],
   [14.41699131840608, 50.087380815829185],
   [14.389216586346102, 50.08929063934356],
   [14.411521238438702, 50.08571655883016],
   [14.38984784072902, 50.0911379161301],
   [14.413049215441427, 50.07747016699747],
   [14.415483236762713, 50.08390281946004],
   [14.397748171806947, 50.08772677482758],
   [14.435106794562088, 50.09387965661264],
   [14.397465877051735, 50.091393211095536],
   [14.428492912477747, 50.08105576712755],
   [14.385572739034965, 50.08561088938783],
   [14.423785832640057, 50.07858886944261],
   [14.39822826386794, 50.080748168783884],
   [14.412169609834518, 50.08011917650889],
   [14.42004008259051, 50.0849938339901],
   [14.411993324745945, 50.0831930418715],
   [14.41128695821139, 50.081543262719116],
   [14.41269661415528, 50.08229587803106],
   [14.42667975141251, 50.07919727641622],
   [14.4

In [99]:
rows = []
for distr_info in shops_coords:
    for i, j in zip(distr_info['shops_coords'], distr_info['shops_uid']):
        rows.append((distr_info['id'], i, j))
csv_dump(rows, 'shops_coords_dist.csv', delimiter=";")